- 서버를 실행하면 상품목록과 성분목록 데이터를 DB에 입력한다.
    - 테이블 구조는 자유롭게 설계
- JSON에 포함된 imageId를 통해 image의 url을 만들어낼 수 있다.
    - base_url : [https://grepp-programmers-challenges.s3.ap-northeast-2.amazonaws.com/2020-birdview/](https://grepp-programmers-challenges.s3.ap-northeast-2.amazonaws.com/2020-birdview/)
    - full : https://grepp-programmers-challenges.s3.ap-northeast-2.amazonaws.com/2020-birdview/image/1e0396a3-71b9-4cb1-b545-c1e38083c838.jpg
    - thumbnail : https://grepp-programmers-challenges.s3.ap-northeast-2.amazonaws.com/2020-birdview/thumbnail/1e0396a3-71b9-4cb1-b545-c1e38083c838.jpg
- 주어진 데이터를 활용해 API를 구성해라
    

In [1]:
from flask import Flask, g, request, Response, make_response, jsonify
import sqlite3

app = Flask(__name__)
app.debug = True

## 상품 데이터 DB에 넣기

일단 주어진 '상품 데이터'와 '성분 데이터'를 Sqlite3 형태의 DB에 저장한다.

In [52]:
import os
import sqlite3
import json

target_path = os.path.join('../hwahaeFlask/static/data')

print(os.listdir(target_path))

conn = sqlite3.connect('../hwahaeFlask/static/data/hwahae.db')
cursor = conn.cursor()
cursor.execute("CREATE TABLE IF NOT EXISTS ingredient (name varchar(50), oily varchar(3), dry varchar(3), sensitive varchar(3))")
cursor.execute("CREATE TABLE IF NOT EXISTS item \
               (id int, \
               imageId varchar(200), \
               name varchar(50), \
               price int, \
               gender varchar(50), \
               category varchar(50), \
              ingredients varchar(200), \
              monthlySales int \
              )")

file_full_path = "../hwahaeFlask/static/data/ingredient-data.json"
with open(file_full_path, encoding = 'utf-8') as f:
    content = f.readline()
    json_content = json.loads(content)
    for document in json_content:
#         for key in document:
#             if document[key] == "":
#                 document[key] = "-"
        
        query = """ Insert into ingredient VALUES("{}", "{}", "{}", "{}")""".format(document['name'], document['oily'], document['dry'], document['sensitive'])
        cursor.execute(query)

    
file_full_path = "../hwahaeFlask/static/data/item-data.json"
with open(file_full_path, encoding = 'utf-8') as f:
    content = f.readline()
    json_content = json.loads(content)
    for document in json_content:
#         for key in document:
#             if document[key] == "":
#                 document[key] = "-"
        
        query = """ Insert into item VALUES({}, "{}", "{}", {},
        "{}", "{}", "{}", {})""".format(
            document['id'], document['imageId'], document['name'], document['price'],
        document['gender'], document['category'], document['ingredients'], document['monthlySales'])
        cursor.execute(query)

conn.commit()
conn.close()

['hwahae.db', 'ingredient-data.json', 'item-data.json']


## 상품 목록 조회하기

- 상품 목록 조회하기
- 피부 타입 별로 상품 목록을 **필터링** 한다.
        - 주어진 피부 타입에 대해 성분 점수를 계산해 높은 상품 순으로 보여준다. 점수가 같다면 낮은 가격을 먼저 표시한다.
        - 50개 단위로 페이징한다. 인자로 페이지 번호가 주어지면, 해당되는 상품 목록을 준다.
        - 상품 카테고리를 선택할 수 있다.
        - 제외되어야 하는 성분들을 지정할 수 있다.
            - `exclude_ingredient` 인자로 전달한 성분**들**을 모두 가지지 않는 상품들만 목록에 포함한다.
        - 포함해야 하는 성분들을 지정할 수 있다.
            - `include_ingredient` 인자로 전달한 성분들을 모두 가지고 있는 상품들만 목록에 포함한다.
- URL : /products

- Method : GET
- Header Ex : `GET /products?skin_type=dry
Content-Type: application/json`

-sample call : /products?skin_type=oily&category=skincare&page=3&include_ingredient=Glycerin

- succes response : [
    {
      "id": 17,
      "imgUrl": "https://grepp-programmers-challenges.s3.ap-northeast-2.amazonaws.com/2020-birdview/thumbnail/00316276-7d5d-47d5-bfd0-a5181cd7b46b.jpg",
      "name": "화해 에센스 토너",
      "price": 23000,
      "ingredients": "Glycerin,Methyl Gluceth-20,Pulsatilla Koreana Extract,Purified Water",
      "monthlySales": 1682
    },
    {
      "id": 23,
      "imgUrl": "https://grepp-programmers-challenges.s3.ap-northeast-2.amazonaws.com/2020-birdview/thumbnail/00316276-7d5d-47d5-bfd0-a5181cd7b46b.jpg",
      "name": "화해 엔젤 토너",
      "price": 4800,
      "ingredients": "Glycerin,Sodium Hyaluronate,Xanthan Gum,Niacinamide,Orchid Extract",
      "monthlySales": 463
    },


    ...


    {
      "id": 88,
      "imgUrl": "https://grepp-programmers-challenges.s3.ap-northeast-2.amazonaws.com/2020-birdview/thumbnail/00316276-7d5d-47d5-bfd0-a5181cd7b46b.jpg",
      "name": "화해 화이트 브라이트닝 소프너 인리치드",
      "price": 24000,
      "ingredients": "Glycerin,Alcohol,Purified Water,Vinyl Dimethicone,PEG-10 Dimethicone",
      "monthlySales": 4437
    }
]


In [8]:
@app.route('/products', methods=['GET'])
def products():
    query_parameters = request.args
    
    skin_type = query_parameters.get('skin_type')
    category = query_parameters.get('category')
    page = query_parameters.get('page')
    exclude_ingredient = query_parameters.get('exclude_ingredient')
    include_ingredient = query_parameters.get('include_ingredient')
    
    query = "SELECT id, imageId, name, price, ingredients, monthlySales FROM item WHERE "
    to_filter = []
    
    #category
    if category:
        query += "category=? AND"
        to_filter.append(category)
    
    #include_ingredient
    if include_ingredient:
        for ingredient in include_ingredient.split(','):
            query += "ingredients LIKE ? AND"
            to_filter.append(ingredient)
    
    #exclude_ingredient
    if exclude_ingredient:
        for ingredient in exclude_ingredient.split(','):
            query += "ingredients NOT LIKE ? AND"
            to_filter.append(ingredient)
    
    query = query[:-4] + ';'

    conn = sqlite3.connect('hwahaeFlask/static/data/hwahae.db')
    conn.row_factory = dict_factory
    cur = conn.cursor()

    results = cur.execute(query, to_filter).fetchall()

    for result in results:
        query = "SELECT * FROM ingredient WHERE"
        to_filter = []

        for ingredient in result['ingredients'].split(','):
            query += " name LIKE ? OR"
            to_filter.append(ingredient)
        query = query[:-3] + ';'
        ing = cur.execute(query, to_filter).fetchall()

        score = 0
        for _ in ing:
            if(_[skin_type] == "O"):
                score += 1
            elif(_[skin_type] == "X"):
                score -= 1
            else:
                continue
        result['score'] = score
        
    results = sorted(results, key= lambda x: x['score'], reverse = True)

    #page
    if page:
        results = results[(int(page)-1)*50:int(page)*50]
    
    return jsonify(results)

## 상품 상세 정보 조회하기

- 상품 id로 특정 상품의 상세 정보 조회
- 이미지 id를 base URL과 조합해 상품 이미지를 불러올 수 있는 URL을 보여줌
- 같은 카테고리의 상품 중 상위 3개의 추천 상품 정보를 조회
    - 인자로 받은 피부 타입에 대한 성분 점수가 높은 순서로 추천. 점수가 같다면 가격이 낮은 상품을 먼저 추천
    - 추천 상품 정보는 상품 아이디, 썸네일 이미지 URL, 상품명, 가격을 포함
    
- URL : /product/:id

- Method : GET
- Header Ex : `GET /product/17?skin_type=oily
Content-Type: application/json`

-sample call : /product/17?skin_type=oily


- succes response : [
    {
      "id": 17,
      "imgUrl": "https://grepp-programmers-challenges.s3.ap-northeast-2.amazonaws.com/2020-birdview/image/00316276-7d5d-47d5-bfd0-a5181cd7b46b.jpg",
      "name": "화해 에센스 토너",
      "price": 23000,
      "gender": "all",
      "category": "skincare",
      "ingredients": "Glycerin,Methyl Gluceth-20,Pulsatilla Koreana Extract,Purified Water",
      "monthlySales": 1682
    },
    {
      "id": 23,
      "imgUrl": "https://grepp-programmers-challenges.s3.ap-northeast-2.amazonaws.com/2020-birdview/thumbnail/00316276-7d5d-47d5-bfd0-a5181cd7b46b.jpg",
      "name": "화해 엔젤 토너",
      "price": 33000
    },
    {
      "id": 37,
      "imgUrl": "https://grepp-programmers-challenges.s3.ap-northeast-2.amazonaws.com/2020-birdview/thumbnail/00316276-7d5d-47d5-bfd0-a5181cd7b46b.jpg",
      "name": "화해 화이트 브라이트닝 소프너 인리치드",
      "price": 24800
    },
    {
      "id": 141,
      "imgUrl": "https://grepp-programmers-challenges.s3.ap-northeast-2.amazonaws.com/2020-birdview/thumbnail/00316276-7d5d-47d5-bfd0-a5181cd7b46b.jpg",
      "name": "화해 퍼펙트 스킨케어",
      "price": 14800
    }
]


In [53]:
@app.route('/product/<pid>', methods=['GET'])
def product(pid):  
    final_results = [] 
    skin_type = request.args.get('skin_type')

    query = "SELECT * FROM item WHERE id = ?"

    conn = sqlite3.connect('hwahaeFlask/static/data/hwahae.db')
    conn.row_factory = dict_factory
    cur = conn.cursor()
    target = cur.execute(query, [pid]).fetchall()[0]
    final_results.append(target)

    query = "SELECT id, imageId, category, ingredients, name, price FROM item WHERE category = ?"
    results = cur.execute(query, [target["category"]]).fetchall()

    for result in results:
        query = "SELECT * FROM ingredient WHERE"
        to_filter = []

        for ingredient in result['ingredients'].split(','):
            query += " name LIKE ? OR"
            to_filter.append(ingredient)
        query = query[:-3] + ';'
        ing = cur.execute(query, to_filter).fetchall()

        score = 0
        for _ in ing:
            if(_[skin_type] == "O"):
                score += 1
            elif(_[skin_type] == "X"):
                score -= 1
            else:
                continue
        result['score'] = score
        
    results = sorted(results, key= lambda x: x['score'], reverse = True)
    
    for result in results[:3]:
        result = {k:v for k, v in result.items() if k in ['id', 'imageId', 'name', 'price']}
        print(result)
        final_results.append(result)
        
    return jsonify(final_results)

결국 만들다보니, 공통으로 쓰이는 함수들이 보인다.

공통적인 구조는,

DB에 쿼리를 날리고, 그 후에 Flask Server에서 연산을 하는 구조인데, 

DB에 쿼리파라미터를 싣고 쿼리를 날리는 과정을 하나의 함수로 만들어 놓으면 될 것 같다. 코드의 재사용을 많이 줄일 것 같음.

여기서는 귀찮아서 안함.

또한 `?` 를 통해 sqlite3에 `bind parameter` 작업을 수행할 때는, 반드시 리스트에 넣어야 한다. 안그럼 str의 하나하나 문자열을 한개의 인자로 인식하더라. 이게 mySql에서도 동일한지 확인해볼 필요가 있다.

전반적으로 플라스크 배운 것들을 활용할 수 있는 프로젝트여서 재밌었다.
